In [7]:
import sys
sys.path.append('/Users/maksym/Documents/Files/Heavy/Work/Dynamiq/dynamiq/examples')

In [6]:
!pip3 install /Users/maksym/Documents/Files/Heavy/Work/Dynamiq/dynamiq

Processing /Users/maksym/Documents/Files/Heavy/Work/Dynamiq/dynamiq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dynamiq: filename=dynamiq-0.1.0-py3-none-any.whl size=227967 sha256=6b1928262b9c860d38a671f1937a9be9cd87f17acc301043ec8f2b0d618a12f9
  Stored in directory: /private/var/folders/8k/lg3sbcy50673220wj1njz2z80000gn/T/pip-ephem-wheel-cache-574nmed1/wheels/27/51/54/151d5963cae748a72f250319c889bb5c38d9a3a336033eeb47
Successfully built dynamiq
  Attempting uninstall: dynamiq
    Found existing installation: dynamiq 0.1.0
    Uninstalling dynamiq-0.1.0:
      Successfully uninstalled dynamiq-0.1.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [8]:
import json
from typing import Any

from dynamiq.connections import E2B
from dynamiq.nodes.agents.orchestrators.graph import END, START, GraphOrchestrator
from dynamiq.nodes.agents.orchestrators.graph_manager import GraphAgentManager
from dynamiq.nodes.tools.e2b_sandbox import E2BInterpreterTool
from dynamiq.nodes.types import InferenceMode
from dynamiq.prompts import Message, Prompt
from dynamiq.runnables import RunnableStatus
from llm_setup import setup_llm

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_observation" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:284: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [9]:
llm = setup_llm()
connection_e2b = E2B()

tool_code = E2BInterpreterTool(connection=connection_e2b)
llm = setup_llm(model_provider="gpt", model_name="gpt-4o-mini", temperature=0)

def code_llm(messages, structured_output=True):
    code_sample = {
        "type": "json_schema",
        "json_schema": {
            "strict": True,
            "name": "generate_code_solution",
            "schema": {
                "type": "object",
                "required": ["libraries", "code"],
                "properties": {
                    "libraries": {
                        "type": "string",
                        "description": (
                            "Libraries that have to be installed (coma separated)." " Example: 'pandas,numpy'"
                        ),
                    },
                    "code": {"type": "string", "description": "Code solution to the problem."},
                },
                "additionalProperties": False,
            },
        },
    }

    llm_result = llm.run(
        input_data={},
        prompt=Prompt(
            messages=messages,
        ),
        schema=code_sample if structured_output else None,
        inference_mode=InferenceMode.STRUCTURED_OUTPUT if structured_output else InferenceMode.XML,
    ).output["content"]

    return json.loads(llm_result) if structured_output else llm_result

def generate_code_solution(context: dict[str, Any], **_):
    """
    Generate a code solution
    """

    print("#####CODE GENERATION#####")

    messages = context.get("messages")

    if context.get("reiterate", False):
        messages += [Message(role="user", content="Generate code again taking into account errors. {}")]

    code_solution = code_llm(messages)
    context["solution"] = code_solution

    context["messages"] += [
        Message(
            role="assistant",
            content=f"\n Imports: {code_solution.get('libraries')} \n Code: {code_solution.get('code')}",
        )
    ]

    context["iterations_num"] += 1
    return code_solution

def reflect(context: dict[str, Any], **_):
    print("#####REFLECTING ON ERRORS#####")
    reflections = code_llm(messages=context.get("messages"))
    context["messages"] += [Message(role="assistant", content=f"Here are reflections on the error: {reflections}")]
    return reflections

def validate_code(context: dict[str, Any], **_):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("#####CHECKING#####")
    solution = context["solution"]

    result = tool_code.run(input_data={"python": solution.get("code"), "packages": solution.get("libraries")})
    if result.status == RunnableStatus.SUCCESS:
        print("#####SUCCESSFUL#####")
        successful_message = [
            Message(role="user", content=f"Your code executed successfully {result.output['content']}")
        ]
        context["messages"] += successful_message
        context["reiterate"] = False
    else:
        print("#####FAILED#####")
        error_message = [
            Message(
                role="user",
                content=(
                    f"Your solution failed the code execution test: {result.output['content']}."
                    " Reflect on possible errors."
                ),
            )
        ]
        context["messages"] += error_message
        context["reiterate"] = True

    return result

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:284: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2024-11-07 19:12:06 - WARNING - Python-dotenv could not parse statement starting at line 2
2024-11-07 19:12:06 - WARNING - Python-dotenv could not parse statement starting at line 5
2024-11-07 19:12:06 - WARNING - Python-dotenv could not parse statement starting at line 8
2024-11-07 19:12:06 - WARNING - Python-dotenv could not parse statement starting at line 9
2024-11-07 19:12:06 - WARNING - Python-dotenv could not parse statement starting at line 11
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/litellm/utils.py:17: DeprecationWarning: 'imghdr' is deprecated 

In [10]:

orchestrator = GraphOrchestrator(
    name="Graph orchestrator",
    manager=GraphAgentManager(llm=llm),
    objective="Provide final code that succeed and reflection.",
)

orchestrator.add_node("generate_code", [generate_code_solution])
orchestrator.add_node("validate_code", [validate_code])
orchestrator.add_node("reflect", [reflect])

orchestrator.add_edge(START, "generate_code")
orchestrator.add_edge("generate_code", "validate_code")
orchestrator.add_edge("reflect", "generate_code")

orchestrator.add_conditional_edge(
    "validate_code", ["generate_code", END], lambda x: "reflect" if x["reiterate"] else END
)


In [11]:
result = orchestrator.run(
    input_data={
        "messages": [Message(role="user", content="Make 100 lines of code")],
        "iterations_num": 0,
        "reiterate": False,
    },
    config=None,
)

print("Result:")
print(result.output.get("content"))

2024-11-07 19:12:08 - INFO - Node Graph orchestrator - 022201b2-ff3b-473d-9b5a-0d7e84351cbb: execution started.
2024-11-07 19:12:08 - INFO - Node OpenAI LLM - f04fff8a-d834-465c-9056-9681fba3b677: execution started.


#####CODE GENERATION#####


2024-11-07 19:12:19 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-07 19:12:19 - INFO - Node OpenAI LLM - f04fff8a-d834-465c-9056-9681fba3b677: execution succeeded in 11.5s.
2024-11-07 19:12:19 - INFO - Node code-interpreter_e2b - b0959c0e-45b6-4d72-bc53-72accabca32a: execution started.
2024-11-07 19:12:19 - INFO - Starting process: pip install -qq numpy pandas


#####CHECKING#####


2024-11-07 19:12:19 - INFO - Started process (id: N7aEQTdXaajz)
2024-11-07 19:12:35 - INFO - Handling process exit (id: N7aEQTdXaajz)
2024-11-07 19:12:35 - INFO - Starting process: python3 /home/user/ec999c66894e206fcebfd075a5ffa1c14032b850daf490977df86645348f60df.py
2024-11-07 19:12:35 - INFO - Started process (id: 35dqYi7Gxw8K)
2024-11-07 19:12:36 - INFO - Handling process exit (id: 35dqYi7Gxw8K)
2024-11-07 19:12:36 - ERROR - Node code-interpreter_e2b - b0959c0e-45b6-4d72-bc53-72accabca32a: execution error: Error during Python code execution: Traceback (most recent call last):
  File "/home/user/ec999c66894e206fcebfd075a5ffa1c14032b850daf490977df86645348f60df.py", line 80, in <module>
    pivot_table = merged_df.pivot_table(values='Float', index='Float', columns='Integer', aggfunc='mean')
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pandas/core/frame.py", line 9509, in pivot_t

...

#####CHECKING#####


2024-11-07 19:12:59 - INFO - Started process (id: 8l33rZMPQUv7)
2024-11-07 19:13:07 - INFO - Handling process exit (id: 8l33rZMPQUv7)
2024-11-07 19:13:07 - INFO - Starting process: python3 /home/user/3d8b22340a40b037e56ccf7418b6a976a34cd80401c14d2ac12a4341a124f094.py
2024-11-07 19:13:07 - INFO - Started process (id: 98dTdtufhXyw)
2024-11-07 19:13:09 - INFO - Handling process exit (id: 98dTdtufhXyw)
2024-11-07 19:13:09 - INFO - Node code-interpreter_e2b - b0959c0e-45b6-4d72-bc53-72accabca32a: execution succeeded in 9.6s.
2024-11-07 19:13:09 - INFO - Node Graph Manager - 36c77a31-a947-49cd-a889-ceeb25d85d26: execution started.
2024-11-07 19:13:09 - INFO - AgentManager Graph Manager - 36c77a31-a947-49cd-a889-ceeb25d85d26: started with input {'action': 'final', 'input_task': 'Provide final code that succeed and reflection.', 'chat_history': [{'role': 'user', 'content': 'Provide final code that succeed and reflection.'}, {'role': 'system', 'content': 'Result: {\'libraries\': \'numpy,pandas\

#####SUCCESSFUL#####


2024-11-07 19:13:26 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-07 19:13:26 - INFO - Node OpenAI LLM - f04fff8a-d834-465c-9056-9681fba3b677: execution succeeded in 17.5s.
2024-11-07 19:13:26 - INFO - Node Graph Manager - 36c77a31-a947-49cd-a889-ceeb25d85d26: execution succeeded in 17.5s.
2024-11-07 19:13:26 - INFO - Node Graph orchestrator - 022201b2-ff3b-473d-9b5a-0d7e84351cbb: execution succeeded in 1.3m.


Result:
### Final Report on Data Processing and Analysis

#### Overview
This report summarizes the successful execution of a Python script that generates a random dataset, performs various data manipulations, and visualizes the results using libraries such as NumPy, Pandas, and Matplotlib. The code was developed to demonstrate data handling capabilities, including statistical calculations, data filtering, and visualization techniques.

#### Libraries Used
- **NumPy**: For numerical operations and random data generation.
- **Pandas**: For data manipulation and analysis, including DataFrame creation and operations.
- **Matplotlib**: For data visualization through plots.

#### Code Execution Summary
The following steps were executed in the code:

1. **Data Generation**:
   - A random dataset with 100 rows and 5 columns was generated using NumPy. Each entry is a floating-point number between 0 and 1.

2. **DataFrame Creation**:
   - A Pandas DataFrame was created from the random data, with